In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [2]:
df = pd.read_csv('train.csv')

In [3]:
#remove kms from kms_driven
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
# Remover vírgula e espaço em branco da coluna 'kms_driven'
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

In [4]:
# Change the data types of multiple columns
df = df.astype({
    
    'kms_driven': float,
    
})

In [5]:
# Check data types
data_types = df.dtypes
print("Data type by column:")
print(data_types)

Data type by column:
full_name             object
registered_year       object
engine_capacity       object
insurance             object
transmission_type     object
kms_driven           float64
owner_type            object
fuel_type             object
max_power             object
seats                float64
mileage               object
body_type             object
city                  object
resale_price_Lakh    float64
dtype: object


In [6]:
#elina brancos ou NAN quando é float
df = df.dropna(subset=['kms_driven'])

In [7]:
# define os limites dos grupos
limites = [0, 50000, 150000, 300000, 600000, 900000, float('inf')]  
rotulos = ['Até 50.000 km', '50.000 km a 150.000 km', '150.000 km a 300.000 km', '300.000 km a 600.000 km', '600.000 km a 900.000 km', 'Acima de 900.000 km']

# cria a nova coluna
df['kms_group'] = pd.cut(df['kms_driven'], bins=limites, labels=rotulos, right=False)

# Incluir a linha com 'kms_driven' igual a 1200000 na última categoria
df.loc[df['kms_driven'] == 1200000, 'kms_group'] = 'Acima de 900.000 km'

print(df)


                                               full_name registered_year  \
0                                2019 Tata Tiago XZ Plus        Nov 2019   
1                              2018 Honda WR-V i-VTEC VX        Apr 2018   
2                              2015 Honda City i VTEC SV        May 2015   
3      2021 Tata New Safari XZA Plus Adventure Editio...            2021   
4                        2019 Maruti Baleno 1.2 CVT Zeta        Jun 2019   
...                                                  ...             ...   
13951                          2021 Kia Carnival Premium            2021   
13952               2017 Hyundai i20 1.2 Magna Executive        Dec 2017   
13953             2017 Skoda Rapid 1.5 TDI Ambition BSIV        Jan 2018   
13954                      2017 Maruti Vitara Brezza ZDi        Apr 2017   
13955                         2015 Honda Amaze S i-Vtech        Mar 2015   

      engine_capacity              insurance transmission_type  kms_driven  \
0        

In [8]:
# unique 'kms_group'
valores_unicos_kms_group= df['kms_group'].unique()
print(valores_unicos_kms_group)

['Até 50.000 km', '50.000 km a 150.000 km', '150.000 km a 300.000 km', 'Acima de 900.000 km', '600.000 km a 900.000 km', '300.000 km a 600.000 km']
Categories (6, object): ['Até 50.000 km' < '50.000 km a 150.000 km' < '150.000 km a 300.000 km' < '300.000 km a 600.000 km' < '600.000 km a 900.000 km' < 'Acima de 900.000 km']


In [9]:
# Check ID_kms

# Sua coluna de tipos de combustível
kms_group = df['kms_group']

# Número de documentos (ou entradas) no DataFrame
n_docs = len(df)

idf = {}

# Usando drop_duplicates para obter valores únicos
unique_kms_group = kms_group.drop_duplicates()

for km_group in unique_kms_group:
    k = sum(km_group in row for row in df['kms_group'])  # contagem de documentos que contêm este tipo de combustível
    idf[km_group] = np.log10(n_docs / (k + 1))  # Adicionei +1 para evitar divisão por zero

for km_group, value in idf.items():
    print(f'{km_group:>15}: {value:>10}')




  Até 50.000 km: 0.3647105429842729
50.000 km a 150.000 km: 0.25514179755998767
150.000 km a 300.000 km: 1.9116714838390252
Acima de 900.000 km: 3.6675463395115164
600.000 km a 900.000 km: 3.6675463395115164
300.000 km a 600.000 km: 3.6675463395115164


In [10]:
#Create kms_group ID: IDF

# Sua coluna de tipos de combustível
kms_group = df['kms_group']

# Número de documentos (ou entradas) no DataFrame
n_docs = len(df)

idf = {}

# Usando drop_duplicates para obter valores únicos
unique_kms_group = kms_group.drop_duplicates()

for km_group in unique_kms_group:
    k = sum(km_group in row for row in df['kms_group'])  # contagem de documentos que contêm este tipo de combustível
    idf[km_group] = np.log10(n_docs / (k + 1))  # Adicionei +1 para evitar divisão por zero

for km_group, value in idf.items():
    print(f'{km_group:>15}: {value:>10}')

# Criar nova coluna 'idf' no DataFrame
df['id_kms_group'] = df['kms_group'].map(idf)

# Se desejar imprimir o DataFrame com a nova coluna
print(df)

  Até 50.000 km: 0.3647105429842729
50.000 km a 150.000 km: 0.25514179755998767
150.000 km a 300.000 km: 1.9116714838390252
Acima de 900.000 km: 3.6675463395115164
600.000 km a 900.000 km: 3.6675463395115164
300.000 km a 600.000 km: 3.6675463395115164
                                               full_name registered_year  \
0                                2019 Tata Tiago XZ Plus        Nov 2019   
1                              2018 Honda WR-V i-VTEC VX        Apr 2018   
2                              2015 Honda City i VTEC SV        May 2015   
3      2021 Tata New Safari XZA Plus Adventure Editio...            2021   
4                        2019 Maruti Baleno 1.2 CVT Zeta        Jun 2019   
...                                                  ...             ...   
13951                          2021 Kia Carnival Premium            2021   
13952               2017 Hyundai i20 1.2 Magna Executive        Dec 2017   
13953             2017 Skoda Rapid 1.5 TDI Ambition BSIV        